In [10]:
from owlready2 import get_ontology
from pathlib import Path

# Path to your OWL file

onto = get_ontology(r"C:\Users\owola\OneDrive - York St John University\YORKSJ\Artificial Intelligence Concept\Assessment\test_onto.owx").load()

#print(list(cls.name for cls in onto.classes()))


In [15]:
from owlready2 import Thing

Email = onto.Email
FromField = onto.FromField
ReplyToField = onto.ReplyToField
ReturnPathField = onto.ReturnPathField

def analyze_email(from_val, reply_val, return_val):
    # Create email instance
    email = Email("EmailInstance")

    ff = FromField("FF")
    ff.fieldValue = from_val

    rf = ReplyToField("RT")
    rf.fieldValue = reply_val

    rpf = ReturnPathField("RP")
    rpf.fieldValue = return_val

    email.hasField = [ff, rf, rpf]

    # Run reasoner
    with onto:
        try:
            sync_reasoner()
        except:
            pass

    # Indicators based on inferred classes
    indicators = [
        cls.name
        for cls in email.is_a
        if cls.name.endswith("Mismatch") or cls.name.endswith("Spoofing")
    ]

    explanation_map = {
        "DomainMismatch": "The From and Reply-To domains do not match.",
        "ReturnPathMismatch": "The Return-Path domain is different from the From field.",
        "DisplayNameSpoofing": "The display name does not match the sender address."
    }

    return {
        "indicators": indicators,
        "explanations": [explanation_map.get(i, i) for i in indicators],
        "is_phishing": len(indicators) > 0
    }

print(f"Tutor Logic Ready. {Email}")


Tutor Logic Ready. C:\Users\owola\OneDrive - York St John University\YORKSJ\Artificial Intelligence Concept\Assessment\test_onto.owx.Email


In [12]:
from fastapi import FastAPI, Form
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/analyze")
async def analyze(
    from_field: str = Form(...),
    reply_to: str = Form(...),
    return_path: str = Form(...)
):
    result = analyze_email(from_field, reply_to, return_path)
    return JSONResponse(result)

print("FastAPI backend initialized.")


FastAPI backend initialized.


In [18]:
import os
import webbrowser

# Determine base directory
base_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()

# Target folder and file
folder_path = os.path.join(base_dir, "static2")
file_path = os.path.join(folder_path, "index2.html")

# Create folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# HTML content
html_content = """
<!DOCTYPE html>
<html>
<head>
  <title>Phishing Email Tutor</title>
  <style>
    body { font-family: Arial; margin: 40px; }
    input { width: 400px; padding: 8px; }
    button { padding: 10px 20px; margin-top: 20px; }
    #result { margin-top: 30px; }
  </style>
</head>
<body>
  <h1>Intelligent Tutoring System: Phishing Email Analysis</h1>
  
  <form id="analyzeForm">
    <p>From Address:</p>
    <input type="text" id="from_field" required>

    <p>Reply-To Address:</p>
    <input type="text" id="reply_to" required>

    <p>Return-Path Address:</p>
    <input type="text" id="return_path" required>

    <br>
    <button type="submit">Analyze Email</button>
  </form>

  <div id="result"></div>

  <script>
    document.getElementById("analyzeForm").onsubmit = async function(e) {
      e.preventDefault();

      const formData = new FormData();
      formData.append("from_field", document.getElementById("from_field").value);
      formData.append("reply_to", document.getElementById("reply_to").value);
      formData.append("return_path", document.getElementById("return_path").value);

      const res = await fetch("/analyze", {
        method: "POST",
        body: formData
      });

      const data = await res.json();

      document.getElementById("result").innerHTML =
        `<h3>Analysis Result</h3>
         <p><strong>Phishing:</strong> ${data.is_phishing ? "Yes" : "No"}</p>
         <p><strong>Indicators:</strong> ${data.indicators.join(", ")}</p>
         <p><strong>Explanation:</strong></p>
         <ul>${data.explanations.map(e => `<li>${e}</li>`).join("")}</ul>`;
    }
  </script>
</body>
</html>
"""

# Write HTML file
with open(file_path, "w") as f:
    f.write(html_content)

print(f"Frontend Ready: {file_path}")

# Open in default web browser
webbrowser.open(f"file://{file_path}")


Frontend Ready: c:\Users\owola\OneDrive - York St John University\YORKSJ\Artificial Intelligence Concept\Assessment\static2\index2.html


True

In [16]:
from flask import Flask, request, render_template_string

app = Flask(__name__)

html_content = """
<!DOCTYPE html>
<html>
<head>
  <title>Intelligent Tutoring System: Phishing Email Analysis</title>
</head>
<body>
  <h1>Intelligent Tutoring System: Phishing Email Analysis</h1>
  <form method="POST">
    From Address:<br>
    <input type="text" name="from_address"><br>
    Reply-To Address:<br>
    <input type="text" name="reply_to_address"><br>
    Return-Path Address:<br>
    <input type="text" name="return_path_address"><br><br>
    <input type="submit" value="Analyze Email">
  </form>
  {% if result %}
    <h2>Analysis Result:</h2>
    <p>{{ result }}</p>
  {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    result = ""
    if request.method == "POST":
        from_addr = request.form.get("from_address")
        reply_to = request.form.get("reply_to_address")
        return_path = request.form.get("return_path_address")

        # Simple example logic
        if from_addr != reply_to or from_addr != return_path:
            result = "Warning: Possible phishing detected!"
        else:
            result = "Email seems legitimate."

    return render_template_string(html_content, result=result)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\owola\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import uvicorn
import asyncio

asyncio.get_event_loop().run_until_complete(
    uvicorn.Server(
        uvicorn.Config(app, host="0.0.0.0", port=8000)
    ).serve()
)


INFO:     Started server process [20900]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
